## Cacluate The amount of times each team has spent tied, leading, trailing, ect 

In [1]:
## Code taken from original college_hockey workbook folder on 11/7/24

# Start Here

## NOT WORKING, CALULATIONS AND AGGRIGATIONS LOGIC IS WRING> NEEDS TO BE FIXED 11-11-24

In [2]:
### Path to Target Database file
import os
import sqlite3
import pandas as pd

# path to database file using os module
# data\db\2024_Nov_7_SEMI_CLEAN.db
db_path = os.path.join('..', 'data', 'db', '2024_Nov_11_SEMI_CLEAN.db') # Path to most recent cleaned DB File
conn = sqlite3.connect(db_path)

# output_file_path = '../TEMP/lead_Trail_season_aggregate_states.csv' # Original Path - power BI report is tied to tis file location
output_file_path = '../TEMP/Lead_Trail_season_aggregate_stats_11-11.csv'

# verify the connection
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

In [3]:
import pandas as pd

# path to single game file
# path = '../TEMP/single_game_scoring.csv'
# path = '../TEMP/ss_example_3.csv'

def calculate_game_states_dataframe_ot_tie_corrected_v2(file_path):
    data = pd.read_csv(file_path)

    def period_elapsed_to_seconds(period, time_str):
        minutes, seconds = map(int, time_str.split(':'))
        elapsed_seconds = minutes * 60 + seconds
        
        if '1st' in period:
            return elapsed_seconds
        elif '2nd' in period:
            return 20 * 60 + elapsed_seconds
        elif '3rd' in period:
            return 40 * 60 + elapsed_seconds
        else:  # Overtime
            return 60 * 60 + elapsed_seconds

    data['Total_Seconds'] = data.apply(lambda row: period_elapsed_to_seconds(row['Period'], row['Time']), axis=1)

    teams = list(set(data['Home_Team'].unique().tolist() + data['Away_Team'].unique().tolist()))
    team_states = {team: {'Tied': 0, 'Lead by 1': 0, 'Lead by 2': 0, 'Lead by 3+': 0,
                          'Down by 1': 0, 'Down by 2': 0, 'Down by 3+': 0} for team in teams}
    
    current_score = {teams[0]: 0, teams[1]: 0}
    last_timestamp = 0
    game_end = 3 * 20 * 60  # Default game end, will adjust if OT is detected
    overtime_played = False

    first_goal_time = data.sort_values(by='Total_Seconds').iloc[0]['Total_Seconds'] if not data.empty else 0
    team_states[teams[0]]['Tied'] = first_goal_time
    team_states[teams[1]]['Tied'] = first_goal_time
    last_timestamp = first_goal_time

    for index, row in data.sort_values(by='Total_Seconds').iterrows():
        current_score[row['Team']] += 1
        lead_diff = current_score[teams[0]] - current_score[teams[1]]
        time_elapsed = row['Total_Seconds'] - last_timestamp

        if 'Overtime' in row['Period']:
            overtime_played = True
            game_end = row['Total_Seconds']  # Adjust game end to when OT goal is scored

        if lead_diff == 0:
            for team in teams:
                team_states[team]['Tied'] += time_elapsed
        else:
            leading_team, trailing_team = (teams[0], teams[1]) if lead_diff > 0 else (teams[1], teams[0])
            lead_abs = abs(lead_diff)
            update_states = lambda lead: (f'Lead by {min(lead, 3)}+' if lead >= 3 else f'Lead by {lead}',
                                          f'Down by {min(lead, 3)}+' if lead >= 3 else f'Down by {lead}')
            state_lead, state_trail = update_states(lead_abs)
            team_states[leading_team][state_lead] += time_elapsed
            team_states[trailing_team][state_trail] += time_elapsed
        
        last_timestamp = row['Total_Seconds']

    # Determine if game ended in a tie without explicit OT goal and adjust game_end correctly
    lead_diff_final = current_score[teams[0]] - current_score[teams[1]]
    if lead_diff_final == 0 and not overtime_played:
        # Adjust for games that end in a tie without 'OT' period goals
        overtime_played = True
        game_end = 65 * 60  # Adjust game end to include 5 minutes of OT

    final_time_elapsed = game_end - last_timestamp
    if lead_diff_final == 0 and not data.empty:
        for team in teams:
            team_states[team]['Tied'] += final_time_elapsed

    team_states_df = pd.DataFrame.from_dict(team_states, orient='index').reset_index().rename(columns={'index': 'Team'})
    
    return team_states_df

# Re-generate the DataFrame with the corrected function for the new file
# game_states_df_ot_tie_corrected_v2 = calculate_game_states_dataframe_ot_tie_corrected_v2(path)
# game_states_df_ot_tie_corrected_v2


In [4]:
## HOTFIX FOR OLD DB FILE - Add Home_Team and Away_Team to  scoring summary table in the db by spliting the Game_ID
## Home split '-' -1
## Away split '-' -2

# # example # Step 2: Create new columns for Home and Away Teams by parsing Game_ID
# df_game_details['Away_Team'] = df_game_details['Game_ID'].apply(lambda x: x.split('-')[3])
# df_game_details['Home_Team'] = df_game_details['Game_ID'].apply(lambda x: x.split('-')[4])



In [5]:
# season_file_path = '../TEMP/scoring_summary_old.csv'
# season_file_path = '../TEMP/Season_scoring.csv'

# extract scoring table from database
scoring_query = "SELECT * FROM scoring_summary"
scoring_df = pd.read_sql_query(scoring_query, conn)

## HOTFIX FOR OLD DB FILEs - Add Home_Team and Away_Team to  scoring summary table in the db by spliting the Game_ID
scoring_df['Away_Team'] = scoring_df['Game_ID'].apply(lambda x: x.split('-')[3])
scoring_df['Home_Team'] = scoring_df['Game_ID'].apply(lambda x: x.split('-')[4])


from tqdm import tqdm  # For progress tracking




# Function to process each game in the dataset and aggregate game states
def process_season_game_states(season_data):
    # season_data = pd.read_csv(file_path)
    unique_games = season_data['Game_ID'].unique()
    
    # Initialize an empty DataFrame for aggregating results
    season_aggregate_states = pd.DataFrame()
    
    for game_id in tqdm(unique_games, desc="Processing Games"):
        game_data = season_data[season_data['Game_ID'] == game_id]
        
        # Save individual game data to a temporary CSV to use with the existing function
        temp_game_path = '../TEMP/temp_game.csv'
        game_data.to_csv(temp_game_path, index=False)
        
        # Calculate game states for the current game
        game_states_df = calculate_game_states_dataframe_ot_tie_corrected_v2(temp_game_path)
        
        # Aggregate the results
        if season_aggregate_states.empty:
            season_aggregate_states = game_states_df
        else:
            season_aggregate_states = season_aggregate_states.merge(game_states_df, on='Team', how='outer')
            for state in ['Tied', 'Lead by 1', 'Lead by 2', 'Lead by 3+', 'Down by 1', 'Down by 2', 'Down by 3+']:
                season_aggregate_states[state] = season_aggregate_states[f'{state}_x'].fillna(0) + season_aggregate_states[f'{state}_y'].fillna(0)
                season_aggregate_states.drop([f'{state}_x', f'{state}_y'], axis=1, inplace=True)
    
    return season_aggregate_states

# Process the season dataset
season_aggregate_states_df = process_season_game_states(scoring_df)

season_aggregate_states_df

Processing Games: 100%|██████████| 279/279 [00:04<00:00, 62.51it/s]


,Team,Tied,Lead by 1,Lead by 2,Lead by 3+,Down by 1,Down by 2,Down by 3+
0,Air Force,18514.0,4022.0,2541.0,0.0,5528.0,3959.0,6347.0
1,Alaska,12239.0,470.0,2319.0,709.0,6827.0,5262.0,2059.0
2,Alaska Anchorage,7388.0,953.0,637.0,0.0,2633.0,7695.0,6293.0
3,American Int'l,13935.0,1725.0,524.0,0.0,1120.0,6427.0,6209.0
4,Arizona State,14408.0,6217.0,1041.0,2435.0,2445.0,7467.0,2217.0
...,...,...,...,...,...,...,...,...
59,Union,10857.0,6272.0,4522.0,2348.0,940.0,2450.0,1397.0
60,Vermont,15833.0,3178.0,2605.0,2695.0,4113.0,5342.0,0.0
61,Western Michigan,4068.0,3904.0,6218.0,2892.0,1089.0,28.0,0.0
62,Wisconsin,13509.0,2135.0,457.0,0.0,10875.0,4745.0,1594.0


In [6]:
# Sort by Down by 3+ time to find the teams that were trailing by 3+ goals the most
# season_aggregate_stats_df.sort_values(by='Down by 3+', ascending=True).head(10)
season_aggregate_stats_df = season_aggregate_states_df

In [7]:
### Calculate the percentages for the different game states
# Calculate the total game time for each team
season_aggregate_stats_df['Total_Time'] = season_aggregate_stats_df['Tied'] + season_aggregate_stats_df['Lead by 1'] + season_aggregate_stats_df['Lead by 2'] + season_aggregate_stats_df['Lead by 3+'] + season_aggregate_stats_df['Down by 1'] + season_aggregate_stats_df['Down by 2'] + season_aggregate_stats_df['Down by 3+']

# create column of all leads and all deficits
season_aggregate_stats_df['All_Lead'] = season_aggregate_stats_df['Lead by 1'] + season_aggregate_stats_df['Lead by 2'] + season_aggregate_stats_df['Lead by 3+']
season_aggregate_stats_df['All_Deficit'] = season_aggregate_stats_df['Down by 1'] + season_aggregate_stats_df['Down by 2'] + season_aggregate_stats_df['Down by 3+']

# Calculate the percentages for each game state
for state in ['Tied', 'Lead by 1', 'Lead by 2', 'Lead by 3+', 'Down by 1', 'Down by 2', 'Down by 3+', 'All_Lead', 'All_Deficit']:
    season_aggregate_stats_df[f'PCT_{state}'] = season_aggregate_stats_df[state] / season_aggregate_stats_df['Total_Time']

# Display the DataFrame with the calculated percentages
# season_aggregate_stats_df

In [8]:
# Convert all time columns from seconds to minutes
for state in ['Tied', 'Lead by 1', 'Lead by 2', 'Lead by 3+', 'Down by 1', 'Down by 2', 'Down by 3+', 'All_Lead', 'All_Deficit', 'Total_Time']:
    season_aggregate_stats_df[state] = season_aggregate_stats_df[state] / 60

# Display the DataFrame with the times converted to minutes
season_aggregate_stats_df


,Team,Tied,Lead by 1,Lead by 2,Lead by 3+,Down by 1,Down by 2,Down by 3+,Total_Time,All_Lead,All_Deficit,PCT_Tied,PCT_Lead by 1,PCT_Lead by 2,PCT_Lead by 3+,PCT_Down by 1,PCT_Down by 2,PCT_Down by 3+,PCT_All_Lead,PCT_All_Deficit
0,Air Force,308.566667,67.033333,42.350000,0.000000,92.133333,65.983333,105.783333,681.850000,109.383333,263.900000,0.452543,0.098311,0.062110,0.000000,0.135123,0.096771,0.155142,0.160421,0.387035
1,Alaska,203.983333,7.833333,38.650000,11.816667,113.783333,87.700000,34.316667,498.083333,58.300000,235.800000,0.409537,0.015727,0.077597,0.023724,0.228442,0.176075,0.068897,0.117049,0.473415
2,Alaska Anchorage,123.133333,15.883333,10.616667,0.000000,43.883333,128.250000,104.883333,426.650000,26.500000,277.016667,0.288605,0.037228,0.024884,0.000000,0.102856,0.300598,0.245830,0.062112,0.649283
3,American Int'l,232.250000,28.750000,8.733333,0.000000,18.666667,107.116667,103.483333,499.000000,37.483333,229.266667,0.465431,0.057615,0.017502,0.000000,0.037408,0.214663,0.207381,0.075117,0.459452
4,Arizona State,240.133333,103.616667,17.350000,40.583333,40.750000,124.450000,36.950000,603.833333,161.550000,202.150000,0.397681,0.171598,0.028733,0.067209,0.067486,0.206100,0.061192,0.267541,0.334778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Union,180.950000,104.533333,75.366667,39.133333,15.666667,40.833333,23.283333,479.766667,219.033333,79.783333,0.377163,0.217884,0.157090,0.081567,0.032655,0.085111,0.048531,0.456541,0.166296
60,Vermont,263.883333,52.966667,43.416667,44.916667,68.550000,89.033333,0.000000,562.766667,141.300000,157.583333,0.468904,0.094118,0.077149,0.079814,0.121809,0.158206,0.000000,0.251081,0.280015
61,Western Michigan,67.800000,65.066667,103.633333,48.200000,18.150000,0.466667,0.000000,303.316667,216.900000,18.616667,0.223529,0.214517,0.341667,0.158910,0.059838,0.001539,0.000000,0.715094,0.061377
62,Wisconsin,225.150000,35.583333,7.616667,0.000000,181.250000,79.083333,26.566667,555.250000,43.200000,286.900000,0.405493,0.064085,0.013718,0.000000,0.326430,0.142428,0.047846,0.077803,0.516704


In [9]:
## Output the results to a CSV file

season_aggregate_stats_df.to_csv(output_file_path, index=False)